In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("Hello Spark") \
        .master("local[3]") \
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/02/05 16:23:48 WARN Utils: Your hostname, otaviordgs resolves to a loopback address: 127.0.1.1; using 172.24.57.217 instead (on interface eth0)
25/02/05 16:23:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/05 16:23:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
path = "data/online_retail.csv"
df = spark.read.csv(path, inferSchema=True, header=True, mode="FAILFAST")

In [3]:
df.show(10)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [4]:
# 1. Quantos registros existem no dataset?
df.count()

541909

In [6]:
view = df.createOrReplaceTempView("online_retail")
sql_1 = "SELECT COUNT(*) FROM online_retail"
spark.sql(sql_1).show()

+--------+
|count(1)|
+--------+
|  541909|
+--------+



In [5]:
# 2. Quais são as colunas disponíveis no dataset e seus tipos de dados?
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [8]:
# 3. Quantos produtos únicos existem no dataset?
df.select("Description") \
    .distinct() \
    .count()

4224

In [7]:
sql_3 = "SELECT COUNT(DISTINCT Description) FROM online_retail"
spark.sql(sql_3).show()

+---------------------------+
|count(DISTINCT Description)|
+---------------------------+
|                       4223|
+---------------------------+



In [22]:
# 4. Qual é o produto mais vendido (em quantidade)?
from pyspark.sql.functions import *
df.groupBy("Description")\
    .count() \
    .orderBy(desc("count")) \
    .show(1)

+--------------------+-----+
|         Description|count|
+--------------------+-----+
|WHITE HANGING HEA...| 2369|
+--------------------+-----+
only showing top 1 row



In [11]:
sql_4 = """
    SELECT Description, COUNT(*) AS qtd
    FROM online_retail
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 3
"""
spark.sql(sql_4).show(truncate=False)

+----------------------------------+----+
|Description                       |qtd |
+----------------------------------+----+
|WHITE HANGING HEART T-LIGHT HOLDER|2369|
|REGENCY CAKESTAND 3 TIER          |2200|
|JUMBO BAG RED RETROSPOT           |2159|
+----------------------------------+----+



In [8]:
# 5. Qual é o faturamento total da loja?
df.select(sum(expr("UnitPrice * Quantity"))) \
    .show()

+---------------------------+
|sum((UnitPrice * Quantity))|
+---------------------------+
|          9747747.933998296|
+---------------------------+



In [12]:
sql_5 = "SELECT SUM(UnitPrice * Quantity) FROM online_retail"
spark.sql(sql_5).show()

+---------------------------+
|sum((UnitPrice * Quantity))|
+---------------------------+
|          9747747.933998296|
+---------------------------+



In [9]:
# 6. Quais são os cinco países com mais transações
df.groupBy("Country")\
    .count() \
    .orderBy(desc("count")) \
    .show(5)

+--------------+------+
|       Country| count|
+--------------+------+
|United Kingdom|495478|
|       Germany|  9495|
|        France|  8557|
|          EIRE|  8196|
|         Spain|  2533|
+--------------+------+
only showing top 5 rows



In [13]:
sql_6 = """
    SELECT Country, COUNT(*) AS qtd
    FROM online_retail
    GROUP BY 1
    ORDER BY 2 DESC
"""
spark.sql(sql_6).show(5)

+--------------+------+
|       Country|   qtd|
+--------------+------+
|United Kingdom|495478|
|       Germany|  9495|
|        France|  8557|
|          EIRE|  8196|
|         Spain|  2533|
+--------------+------+
only showing top 5 rows



In [10]:
# 7. Qual é a média de preço dos produtos vendidos?
df.select(avg("UnitPrice").alias("average_price")) \
    .show()

+-----------------+
|    average_price|
+-----------------+
|4.611113626086822|
+-----------------+



In [14]:
sql_7 = "SELECT AVG(UnitPrice) FROM online_retail"
spark.sql(sql_7).show()

+-----------------+
|   avg(UnitPrice)|
+-----------------+
|4.611113626086822|
+-----------------+



In [11]:
# 8. Quantas transações ocorreram por mês?
df_8 = df.withColumn("InvoiceDate", to_timestamp(col("InvoiceDate"), "M/d/yyyy H:mm"))
df_8 = df_8.withColumn("invoice_month_year", to_date("InvoiceDate", "MM/yyyy"))
df_8.groupBy("invoice_month_year") \
    .agg(count_distinct("InvoiceNo").alias("num_transactions")) \
    .orderBy(desc("invoice_month_year")) \
    .show(5)

+------------------+----------------+
|invoice_month_year|num_transactions|
+------------------+----------------+
|        2011-12-09|              49|
|        2011-12-08|             145|
|        2011-12-07|             138|
|        2011-12-06|             149|
|        2011-12-05|             151|
+------------------+----------------+
only showing top 5 rows



In [27]:
treated_date: DataFrame = df.withColumn("InvoiceDate", date_trunc("month",to_date("InvoiceDate", "M/d/yyyy H:mm")))
treated_date.createOrReplaceTempView("online_retail")
sql_8 = """
    SELECT InvoiceDate, COUNT(*) AS qtd
    FROM online_retail
    GROUP BY 1
    ORDER BY 2 DESC
"""
spark.sql(sql_8).show()

+-------------------+-----+
|        InvoiceDate|  qtd|
+-------------------+-----+
|2011-11-01 00:00:00|84711|
|2011-10-01 00:00:00|60742|
|2011-09-01 00:00:00|50226|
|2010-12-01 00:00:00|42481|
|2011-07-01 00:00:00|39518|
|2011-05-01 00:00:00|37030|
|2011-06-01 00:00:00|36874|
|2011-03-01 00:00:00|36748|
|2011-08-01 00:00:00|35284|
|2011-01-01 00:00:00|35147|
|2011-04-01 00:00:00|29916|
|2011-02-01 00:00:00|27707|
|2011-12-01 00:00:00|25525|
+-------------------+-----+



In [ ]:
# Intermediarias
# 9. Qual foi o mês com o maior faturamento?
treated_date.groupBy("InvoiceDate") \
    .agg(round(sum(expr("UnitPrice * Quantity")), 2).alias("invoice")) \
    .orderBy(desc("invoice")).show()

+-------------------+---------------+
|        InvoiceDate|invoice_by_date|
+-------------------+---------------+
|2011-11-01 00:00:00|     1461756.25|
|2011-10-01 00:00:00|     1070704.67|
|2011-09-01 00:00:00|     1019687.62|
|2010-12-01 00:00:00|      748957.02|
|2011-05-01 00:00:00|      723333.51|
|2011-06-01 00:00:00|      691123.12|
|2011-03-01 00:00:00|      683267.08|
|2011-08-01 00:00:00|      682680.51|
|2011-07-01 00:00:00|      681300.11|
|2011-01-01 00:00:00|      560000.26|
|2011-02-01 00:00:00|      498062.65|
|2011-04-01 00:00:00|      493207.12|
|2011-12-01 00:00:00|      433668.01|
+-------------------+---------------+



In [36]:
sql_9 = """
    SELECT InvoiceDate, ROUND(SUM(UnitPrice * Quantity), 2) AS invoice
    FROM online_retail
    GROUP BY 1
    ORDER BY 2 DESC
"""
spark.sql(sql_9).show(3)

+-------------------+----------+
|        InvoiceDate|   invoice|
+-------------------+----------+
|2011-11-01 00:00:00|1461756.25|
|2011-10-01 00:00:00|1070704.67|
|2011-09-01 00:00:00|1019687.62|
+-------------------+----------+
only showing top 3 rows



In [39]:
# 10. Quais produtos geraram mais receita (top 10)?
df.groupBy("Description") \
    .agg(round(sum(expr("UnitPrice * Quantity")), 2).alias("total_revenue")) \
    .orderBy(desc("total_revenue")).show(10, truncate=False)


+----------------------------------+-------------+
|Description                       |total_revenue|
+----------------------------------+-------------+
|DOTCOM POSTAGE                    |206245.48    |
|REGENCY CAKESTAND 3 TIER          |164762.19    |
|WHITE HANGING HEART T-LIGHT HOLDER|99668.47     |
|PARTY BUNTING                     |98302.98     |
|JUMBO BAG RED RETROSPOT           |92356.03     |
|RABBIT NIGHT LIGHT                |66756.59     |
|POSTAGE                           |66230.64     |
|PAPER CHAIN KIT 50'S CHRISTMAS    |63791.94     |
|ASSORTED COLOUR BIRD ORNAMENT     |58959.73     |
|CHILLI LIGHTS                     |53768.06     |
+----------------------------------+-------------+
only showing top 10 rows



In [37]:
sql_10 = """
    SELECT Description, ROUND(SUM(UnitPrice * Quantity), 2) AS total_revenue
    FROM online_retail
    GROUP BY 1
    ORDER BY 2 DESC
"""
spark.sql(sql_10).show(10)

+--------------------+-------------+
|         Description|total_revenue|
+--------------------+-------------+
|      DOTCOM POSTAGE|    206245.48|
|REGENCY CAKESTAND...|    164762.19|
|WHITE HANGING HEA...|     99668.47|
|       PARTY BUNTING|     98302.98|
|JUMBO BAG RED RET...|     92356.03|
|  RABBIT NIGHT LIGHT|     66756.59|
|             POSTAGE|     66230.64|
|PAPER CHAIN KIT 5...|     63791.94|
|ASSORTED COLOUR B...|     58959.73|
|       CHILLI LIGHTS|     53768.06|
+--------------------+-------------+
only showing top 10 rows



In [55]:
# 11. Existem valores ausentes no dataset? Quais colunas estão mais afetadas?
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [46]:
# 12. Qual é a distribuição de transações por dia da semana? 
day_of_week: DataFrame = df.withColumn("InvoiceDate", dayofweek(to_date("InvoiceDate", "M/d/yyyy H:mm")))
day_of_week.createOrReplaceTempView("day_of_week")
day_of_week.groupBy("InvoiceDate") \
    .count() \
    .orderBy(desc("InvoiceDate")) \
    .show()

+-----------+------+
|InvoiceDate| count|
+-----------+------+
|          6| 82193|
|          5|103857|
|          4| 94565|
|          3|101808|
|          2| 95111|
|          1| 64375|
+-----------+------+



In [48]:

sql_12 = """
    SELECT InvoiceDate, COUNT(*) AS qtd
    FROM day_of_week
    GROUP BY 1
"""
spark.sql(sql_12).show()

+-----------+------+
|InvoiceDate|   qtd|
+-----------+------+
|          1| 64375|
|          6| 82193|
|          3|101808|
|          5|103857|
|          4| 94565|
|          2| 95111|
+-----------+------+



In [59]:
# 13. Quantas transações envolveram devoluções (quantidade negativa)?
df.select("Quantity") \
    .where("Quantity < 0") \
    .count()

10624

In [49]:
sql_13 = """
    SELECT COUNT(*)
    FROM online_retail 
    WHERE Quantity < 0
"""
spark.sql(sql_13).show()

+--------+
|count(1)|
+--------+
|   10624|
+--------+



In [68]:
# 14. Qual cliente gerou mais receita para a loja?
df.groupBy("CustomerID") \
    .agg(round(sum(expr("UnitPrice * Quantity")), 2).alias("total_spent")) \
    .where("CustomerID is not null") \
    .orderBy(desc("total_spent")).show(10)

+----------+-----------+
|CustomerID|total_spent|
+----------+-----------+
|     14646|  279489.02|
|     18102|  256438.49|
|     17450|  187482.17|
|     14911|  132572.62|
|     12415|  123725.45|
|     14156|  113384.14|
|     17511|   88125.38|
|     16684|   65892.08|
|     13694|    62653.1|
|     15311|   59419.34|
+----------+-----------+
only showing top 10 rows



In [52]:
sql_14 = """
    SELECT CustomerID, ROUND(SUM(UnitPrice * Quantity), 2) AS qtd
    FROM online_retail
    WHERE CustomerID IS NOT NULL
    GROUP BY 1
    ORDER BY 2 DESC
"""
spark.sql(sql_14).show(3)

+----------+---------+
|CustomerID|      qtd|
+----------+---------+
|     14646|279489.02|
|     18102|256438.49|
|     17450|187482.17|
+----------+---------+
only showing top 3 rows



In [72]:
# 15. Qual foi o preço médio de produtos comprados por país?
df.groupBy("Country") \
    .agg(avg("UnitPrice").alias("avg_spent")) \
    .orderBy(desc("avg_spent")) \
    .show()

+------------------+------------------+
|           Country|         avg_spent|
+------------------+------------------+
|         Singapore|109.64580786026201|
|         Hong Kong| 42.50520833333332|
|          Portugal| 8.582975641869636|
|            Cyprus| 6.302363344051448|
|            Canada|6.0303311258278125|
|            Norway| 6.012025782688759|
|              EIRE|5.9110773548073245|
|           Finland| 5.448705035971227|
|           Lebanon| 5.387555555555553|
|             Malta| 5.244173228346455|
|            France| 5.028864087881303|
|             Spain|  4.98754441373863|
|   Channel Islands| 4.932124010554094|
|            Greece| 4.885547945205479|
|             Italy|4.8311207970112156|
|European Community| 4.820491803278688|
|           Bahrain| 4.556315789473684|
|    United Kingdom|4.5324221741417015|
|            Brazil| 4.456250000000001|
|               RSA| 4.277586206896552|
+------------------+------------------+
only showing top 20 rows



In [56]:
sql_15 = """
    SELECT Country, AVG(UnitPrice) AS avg_sepnt
    FROM online_retail
    GROUP BY 1
    ORDER BY 2 DESC
"""
spark.sql(sql_15).show()

+------------------+------------------+
|           Country|         avg_sepnt|
+------------------+------------------+
|         Singapore|109.64580786026201|
|         Hong Kong| 42.50520833333332|
|          Portugal| 8.582975641869636|
|            Cyprus| 6.302363344051448|
|            Canada|6.0303311258278125|
|            Norway| 6.012025782688759|
|              EIRE|5.9110773548073245|
|           Finland| 5.448705035971227|
|           Lebanon| 5.387555555555553|
|             Malta| 5.244173228346455|
|            France| 5.028864087881303|
|             Spain|  4.98754441373863|
|   Channel Islands| 4.932124010554094|
|            Greece| 4.885547945205479|
|             Italy|4.8311207970112156|
|European Community| 4.820491803278688|
|           Bahrain| 4.556315789473684|
|    United Kingdom|4.5324221741417015|
|            Brazil| 4.456250000000001|
|               RSA| 4.277586206896552|
+------------------+------------------+
only showing top 20 rows

